In [1]:
from var_func import *
from io_func import *

In [20]:
supply_user = {
    'Provinsi - Industri':'provinsi_industri',
    'Provinsi - Lapangan Usaha':'provinsi_lapus',
    'Pulau - Industri':'pulau_industri',
    'Pulau - Lapangan Usaha':'pulau_lapus',
    'Industri':'industri',
    'Pulau':'pulau',
    'Provinsi':'provinsi',
    'Lapangan Usaha':'lapus'
}
supply_user_opt = [{'label':key, 'value':item} for key,item in supply_user.items()]

app = JupyterDash(__name__)

app.layout = html.Div([
    html.H1('Selamat Datang di VIRIO_2016'),
    html.Hr(),
    html.H3('Pilih Unit Penyedia dan Unit Pengguna'),
    html.Div([
        html.Div([
            html.Label([
                "Penyedia",
                dcc.Dropdown(
                    id="supply-unit", clearable = False,
                    options = supply_user_opt, value = supply_user_opt[0]['value']
                )
            ])
        ], style = {'display':'inline-block', 'width':'49%'}),
        html.Div([
            html.Label([
                "Pengguna",
                dcc.Dropdown(
                    id="user-unit", clearable = False,
                    options = supply_user_opt, value = supply_user_opt[0]['value']
                )
            ])
        ], style = {'display':'inline-block', 'width':'49%'})
    ]),
    html.Hr(),
    html.Div([
        html.Div([
            html.Div([
#                 html.P(id="supply-label", style = {'font-weigth':'bolder', 'font-size':'20px'}),
                dcc.Dropdown(
                    id="supply-name", clearable = False
                )
            ])
        ],style = {'display':'inline-block', 'width':'33.33%'}),
        html.Div([
            html.Div([
                dcc.RadioItems(
                    options = [
                        {'label':'Proporsi output yang digunakan', 'value':'output'},
                        {'label':'Proporsi penggunaan input', 'value':'input'},
                    ],
                    value = 'output',
                    labelStyle = {'display':'block', 'margin-left':'auto', 'margin-right':'auto'}
                )
            ])
        ],style = {'display':'inline-block', 'width':'33.33%'}),
        html.Div([
            html.Div([
#                 html.P(id="user-label", style = {'font-weigth':'bolder', 'font-size':'20px'}),
                dcc.Dropdown(
                    id="user-name", clearable = False
                )
            ])
        ],style = {'display':'inline-block', 'width':'33.33%'}),
    ]),
    html.Hr(),
    html.Hr(),
    html.H1(id="heading-1"),
    html.H2(id="heading-2"),
    dcc.Graph(id="io-used-graph")
])

@app.callback(
#     Output('supply-label','children'),
    Output('supply-name','options'),
    Output('supply-name','value'),
    Input('supply-unit','value')
)
def update_supply_dropdown(input_):
    if len(input_.split('_')) > 1:
        input__ = input_.split('_')
        labels = [
            f'{unit1} - {unit2}' for unit1, unit2 in zip(
                tables[f"kode_{input__[0]}"]["data"][tables[f"kode_{input__[0]}"]["attr"]["nama"]].to_list(),
                tables[f"kode_{input__[1]}"]["data"][tables[f"kode_{input__[1]}"]["attr"]["nama"]].to_list()
            )
        ]
        values = [
            f'{unit1};{unit2}' for unit1, unit2 in zip(
                tables[f"kode_{input__[0]}"]["data"][tables[f"kode_{input__[0]}"]["attr"]["kode"]].to_list(),
                tables[f"kode_{input__[1]}"]["data"][tables[f"kode_{input__[1]}"]["attr"]["kode"]].to_list()
            )
        ]
        
    else :
        labels = tables[f"kode_{input_}"]["data"][tables[f"kode_{input_}"]["attr"]["nama"]].to_list()
        values = tables[f"kode_{input_}"]["data"][tables[f"kode_{input_}"]["attr"]["kode"]].to_list()
        
    options = [{'label':label, 'value':value} for label, value in zip(labels, values)]    
    return options, options[0]['value']

@app.callback(
#     Output('user-label','children'),
    Output('user-name','options'),
    Output('user-name','value'),
    Input('user-unit','value')
)
def update_user_dropdown(input_):
    print(input_.split('_'))
    if len(input_.split('_')) > 1:
        input__ = input_.split('_')
        labels = [
            f'{unit1} - {unit2}' for unit1, unit2 in zip(
                tables[f"kode_{input__[0]}"]["data"][tables[f"kode_{input__[0]}"]["attr"]["nama"]].to_list(),
                tables[f"kode_{input__[1]}"]["data"][tables[f"kode_{input__[1]}"]["attr"]["nama"]].to_list()
            )
        ]
        values = [
            f'{unit1};{unit2}' for unit1, unit2 in zip(
                tables[f"kode_{input__[0]}"]["data"][tables[f"kode_{input__[0]}"]["attr"]["kode"]].to_list(),
                tables[f"kode_{input__[1]}"]["data"][tables[f"kode_{input__[1]}"]["attr"]["kode"]].to_list()
            )
        ]
        
    else :
        labels = tables[f"kode_{input_}"]["data"][tables[f"kode_{input_}"]["attr"]["nama"]].to_list()
        values = tables[f"kode_{input_}"]["data"][tables[f"kode_{input_}"]["attr"]["kode"]].to_list()
        
    options = [{'label':label, 'value':value} for label, value in zip(labels, values)]    
    return options, options[0]['value']

# @app.callback(
#     Output('heading-1', 'children'),
#     Output('heading-2', 'children'),
#     Output('io-used-graph', 'figure'),
#     Input()
# )

    
app.run_server(mode = 'external', debug = True)

Dash app running on http://127.0.0.1:8050/
['provinsi', 'industri']


In [13]:
app._terminate_server_for_port("localhost", 8050)

In [32]:
cond = {'provinsi_penyedia':'32'}
conn = get_connection()
data = change_code(
    pd.read_sql(io_used_per_unit('provinsi', 'provinsi_lapus', 'output', cond), conn, params = cond), 
    {
        'provinsi_penyedia':'provinsi',
        'provinsi_pengguna':'provinsi',
        'lapangan_usaha_pengguna':'lapus'
    }
)
conn.close()

In [33]:
px.bar(data.head(15).sort_values('nilai'), x = 'nilai', y = 'lapangan_usaha_pengguna', color = 'provinsi_pengguna')